In [28]:
import requests
import datetime
from datetime import datetime as dt
import patoolib
import os
import pandas as pd
import sqlalchemy 
import psycopg2
from sqlalchemy import create_engine
import numpy as np
from datetime import timedelta
import os.path
from datetime import datetime
#from sqlalchemy.sql import text

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
for k in ['DEIS']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema DEIS exists, will not be created


In [5]:
now = dt.now()
delta=0
status=400
while status!=200:
    date=now- timedelta(days=delta)
    print('testing date:'+str(date))
    dia=date.strftime("%d")
    mes=date.strftime("%m")
    year=str(date.year)
    url='http://deis.minsal.cl/wp-content/uploads/'+year+'/'+mes+'/DEFUNCIONES_FUENTE_DEIS_2016_'+year+'_'+dia+mes+year+'.rar'
    print('url: '+url)
    path='/data/ETLcache/'
    file=url.split(mes+'/')[1]


    if os.path.isfile(path+file):
        print ("Date exists on archive")
        flag=False
        break
    else:
        req = requests.get(url)
        status=req.status_code
        print(status)
        if status==200:
            print('success')
            print ("Creating file "+path+file)
            open(path+file, 'wb').write(req.content)
            flag=True
        else:
            print('failure, trying previous date')
            delta+=1

testing date:2021-02-17 03:11:35.727587
url: http://deis.minsal.cl/wp-content/uploads/2021/02/DEFUNCIONES_FUENTE_DEIS_2016_2021_17022021.rar
404
failure, trying previous date
testing date:2021-02-16 03:11:35.727587
url: http://deis.minsal.cl/wp-content/uploads/2021/02/DEFUNCIONES_FUENTE_DEIS_2016_2021_16022021.rar
404
failure, trying previous date
testing date:2021-02-15 03:11:35.727587
url: http://deis.minsal.cl/wp-content/uploads/2021/02/DEFUNCIONES_FUENTE_DEIS_2016_2021_15022021.rar
404
failure, trying previous date
testing date:2021-02-14 03:11:35.727587
url: http://deis.minsal.cl/wp-content/uploads/2021/02/DEFUNCIONES_FUENTE_DEIS_2016_2021_14022021.rar
404
failure, trying previous date
testing date:2021-02-13 03:11:35.727587
url: http://deis.minsal.cl/wp-content/uploads/2021/02/DEFUNCIONES_FUENTE_DEIS_2016_2021_13022021.rar
404
failure, trying previous date
testing date:2021-02-12 03:11:35.727587
url: http://deis.minsal.cl/wp-content/uploads/2021/02/DEFUNCIONES_FUENTE_DEIS_2016_20

In [6]:
if not os.path.exists(path+file.rsplit('.rar')[0]):
    print('unpacking RAR file')
    os.mkdir(path+file.rsplit('.rar')[0])
    patoolib.extract_archive(path+file, outdir=path+file.rsplit('.rar')[0])
else:
    print(path+file+' already in archive, no unpacking')

/data/ETLcache/DEFUNCIONES_FUENTE_DEIS_2016_2021_11022021.rar already in archive, no unpacking


In [7]:
current_year = dt.now().year

In [8]:
DBlast_date=cnx.execute('SELECT max("FECHA_DEF") FROM "DEIS"."DEIS_'+str(current_year)+'";')
DBlast_date=DBlast_date.first()[0]

In [9]:
col_names=cnx.execute('SELECT "column_name" FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = '+"'DEIS_"+str(current_year)+"' ;")
col_names = ( col_names.fetchall() )
col_names=[x[0] for x in col_names]
col_names=list(set(col_names)-set(['Metropolitana','FECHA_STR','COVID','ANO_DEF','index']))

In [10]:
col_names

['GLOSA_SUBCATEGORIA_DIAG1',
 'CODIGO_SUBCATEGORIA_DIAG2',
 'GLOSA_CATEGORIA_DIAG1',
 'DIAG1',
 'GLOSA_CAPITULO_DIAG2',
 'CODIGO_GRUPO_DIAG2',
 'GLOSA_COMUNA_RESIDENCIA',
 'DIAG2',
 'CODIGO_GRUPO_DIAG1',
 'GLOSA_CAPITULO_DIAG1',
 'GLOSA_CATEGORIA_DIAG2',
 'GLOSA_REG_RES',
 'CAPITULO_DIAG2',
 'GLOSA_SEXO',
 'CODIGO_COMUNA_RESIDENCIA',
 'CODIGO_CATEGORIA_DIAG2',
 'CODIGO_CATEGORIA_DIAG1',
 'GLOSA_GRUPO_DIAG2',
 'EDAD_CANT',
 'CODIGO_SUBCATEGORIA_DIAG1',
 'CAPITULO_DIAG1',
 'EDAD_TIPO',
 'FECHA_DEF',
 'GLOSA_SUBCATEGORIA_DIAG2',
 'GLOSA_GRUPO_DIAG1']

In [11]:
DBlast_date

datetime.datetime(2021, 2, 3, 0, 0)

In [12]:
file_last_date=file.rsplit('.rar')[0].rsplit('_',1)[1]
file_last_date=file_last_date[:2]+'-'+file_last_date[2:4]+'-'+file_last_date[4:]
file_last_date=datetime.strptime(file_last_date, '%d-%m-%Y')

In [13]:
file_last_date

datetime.datetime(2021, 2, 11, 0, 0)

In [14]:
db_flag=file_last_date>DBlast_date

In [15]:
db_flag

True

In [22]:
def csvs_to_sql(deis,headers):
    schema='DEIS'
    headers=headers['Nombre de la variable']
    headers_diff=deis.columns.shape[0]-len(headers)
    print(headers_diff)
    deis=deis[deis.columns[headers_diff:]]
    #headers=headers[1:]
    deis.columns=headers
    deis['Metropolitana']=0
    deis.loc[deis['GLOSA_REG_RES']=='Metropolitana de Santiago','Metropolitana']=1
    deis['Metropolitana']=deis['Metropolitana'].replace({0:'Resto de Chile',1:'Región Metropolitana'})
    #deis['FECHA_DEF_OR']=deis['FECHA_DEF']
    deis['FECHA_DEF']=pd.to_datetime(deis['FECHA_DEF'],format='%Y-%m-%d',errors='coerce')
    deis['FECHA_DEF']=deis['FECHA_DEF'].dt.to_pydatetime()
    deis['FECHA_STR']=deis['FECHA_DEF'].dt.strftime('%d-%m-%Y')
    deis['COVID']=deis['DIAG1'].replace({'U071':'Covid PCR+','U072':'Covid Sospechoso'})
    deis.loc[~deis['DIAG1'].isin(['U071','U072']),'COVID']=np.nan

    if not 'ANO_DEF' in deis.columns:
        print('creando columna: ANO_DEF')
        deis['ANO_DEF']=deis['FECHA_DEF'].dt.year
        
    for y in deis['ANO_DEF'].unique():
        name='DEIS_'+str(y).split('.')[0]
        df=deis[deis['ANO_DEF']==y]
        if y==current_year:
            print("creating table "+name+' ,schema: '+schema)
            df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
        else:
            try:
                print("creating table (not overwrite mode)"+name+' ,schema: '+schema)
                df.to_sql(name, schema=schema,con=cnx,if_exists='fail')
            except:
                print('table '+name+' ,schema: '+schema+' already exists')
                
            
    deis_covid=deis[deis['COVID'].notnull()]
    name='DEIS_COVID'
    print("creating table "+name+' ,schema: '+schema)
    deis_covid.to_sql(name, schema=schema,con=cnx,if_exists='replace')
    
    edad_fecha=deis.loc[(deis['ANO_DEF']==2020) & (deis['COVID'].notnull())]
    edad_fecha=edad_fecha.groupby(['FECHA_DEF','EDAD_CANT'])['COVID'].count().reset_index()
    edad_fecha=edad_fecha.set_index('FECHA_DEF').groupby('EDAD_CANT').rolling('7d',min_periods=1)['COVID'].mean()
    edad_fecha=edad_fecha.reset_index()
    name='DEIS_EDAD_FECHA_ROLLING'
    print("creating table "+name+' ,schema: '+schema)
    edad_fecha.to_sql(name, schema=schema,con=cnx,if_exists='replace')
    
    return deis

In [ ]:
if db_flag==True:
    print('Reading files')
    for f in os.listdir(path+file.rsplit('.rar')[0]):
        if '.csv' in f:
            deis=pd.read_csv(path+file.rsplit('.rar')[0]+'/'+f,sep=';',encoding='latin1',header=0)
        if '.xlsx' in f:
            headers=pd.read_excel(path+file.rsplit('.rar')[0]+'/'+f,header=3,index_col=1)
        if '.xslx' not in f:
            headers={'Nombre de la variable':col_names}
    deis=csvs_to_sql(deis,headers)
else:
    print('Files already loaded into DBASE, not reading')

Reading files


In [18]:
deis

,2021,2021-01-21,Mujer,1,89,10202,Ancud,De Los Lagos,C20X,C00-D48,...,TUMOR MALIGNO DEL RECTO,,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,2021,2021-01-05,Mujer,1,94,5109,Viña del Mar,De Valparaíso,C229,C00-D48,...,TUMOR MALIGNO DEL HÍGADO NO ESPECIFICADO,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,2021-01-19,Hombre,1,95,13403,Calera de Tango,Metropolitana de Santiago,C61X,C00-D48,...,TUMOR MALIGNO DE LA PRÓSTATA,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,2021-01-22,Hombre,1,96,9120,Villarrica,De La Araucanía,N179,N00-N99,...,INSUFICIENCIA RENAL AGUDA NO ESPECIFICADA,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,2021-01-16,Hombre,1,89,5506,Nogales,De Valparaíso,I509,I00-I99,...,INSUFICIENCIA CARDIACA NO ESPECIFICADA,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,2021-01-22,Hombre,1,104,5501,Quillota,De Valparaíso,J441,J00-J99,...,ENFERMEDAD PULMÓNAR OBSTRUCTIVA CRONICA CON EX...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566539,2016,2016-08-28,Mujer,1,74,10103,Cochamó,De Los Lagos,E119,E00-E90,...,DIABETES MELLITUS NO INSULINODEPENDIENTE SIN M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566540,2016,2016-08-30,Mujer,1,86,10209,Quemchi,De Los Lagos,J449,J00-J99,...,ENFERMEDAD PULMÓNAR OBSTRUCTIVA CRONICA NO ESP...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566541,2016,2016-08-26,Hombre,1,84,10205,Dalcahue,De Los Lagos,C61X,C00-D48,...,TUMOR MALIGNO DE LA PRÓSTATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566542,2016,2016-09-02,Mujer,1,90,10205,Dalcahue,De Los Lagos,J459,J00-J99,...,ASMA NO ESPECIFICADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
deis

,GLOSA_CATEGORIA_DIAG1,GLOSA_SUBCATEGORIA_DIAG1,index,CAPITULO_DIAG2,CAPITULO_DIAG1,CODIGO_GRUPO_DIAG2,DIAG1,GLOSA_CAPITULO_DIAG2,GLOSA_CATEGORIA_DIAG2,GLOSA_SUBCATEGORIA_DIAG2,...,GLOSA_GRUPO_DIAG1,GLOSA_CAPITULO_DIAG1,CODIGO_CATEGORIA_DIAG2,GLOSA_SEXO,GLOSA_GRUPO_DIAG2,Metropolitana,FECHA_DEF_OR,FECHA_STR,COVID,ANO_DEF
0,2021,2021-01-05,Mujer,1,94,5109,Viña del Mar,De Valparaíso,C229,C00-D48,...,NaN,NaN,NaN,NaN,NaN,Resto de Chile,Tumor maligno del hígado y de las vías biliare...,NaN,NaN,NaN
1,2021,2021-01-19,Hombre,1,95,13403,Calera de Tango,Metropolitana de Santiago,C61X,C00-D48,...,NaN,NaN,NaN,NaN,NaN,Resto de Chile,Tumor maligno de la próstata,NaN,NaN,NaN
2,2021,2021-01-22,Hombre,1,96,9120,Villarrica,De La Araucanía,N179,N00-N99,...,NaN,NaN,NaN,NaN,NaN,Resto de Chile,Insuficiencia renal aguda,NaN,NaN,NaN
3,2021,2021-01-16,Hombre,1,89,5506,Nogales,De Valparaíso,I509,I00-I99,...,NaN,NaN,NaN,NaN,NaN,Resto de Chile,Insuficiencia cardíaca,NaN,NaN,NaN
4,2021,2021-01-22,Hombre,1,104,5501,Quillota,De Valparaíso,J441,J00-J99,...,NaN,NaN,NaN,NaN,NaN,Resto de Chile,Otras enfermedades pulmonares obstructivas cró...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566539,2016,2016-08-28,Mujer,1,74,10103,Cochamó,De Los Lagos,E119,E00-E90,...,NaN,NaN,NaN,NaN,NaN,Resto de Chile,Diabetes mellitus no insulinodependiente,NaN,NaN,NaN
566540,2016,2016-08-30,Mujer,1,86,10209,Quemchi,De Los Lagos,J449,J00-J99,...,NaN,NaN,NaN,NaN,NaN,Resto de Chile,Otras enfermedades pulmonares obstructivas cró...,NaN,NaN,NaN
566541,2016,2016-08-26,Hombre,1,84,10205,Dalcahue,De Los Lagos,C61X,C00-D48,...,NaN,NaN,NaN,NaN,NaN,Resto de Chile,Tumor maligno de la próstata,NaN,NaN,NaN
566542,2016,2016-09-02,Mujer,1,90,10205,Dalcahue,De Los Lagos,J459,J00-J99,...,NaN,NaN,NaN,NaN,NaN,Resto de Chile,Asma,NaN,NaN,NaN


In [18]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_DEIS.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/DEIS.ipynb')

0